# Clone kgan repository.

In [1]:
!git clone https://github.com/look4pritam/kgan.git

Cloning into 'kgan'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 21), reused 26 (delta 8), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [2]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Jul 22 07:49 .
drwxr-xr-x 1 root root 4096 Jul 22 07:45 ..
drwxr-xr-x 1 root root 4096 Jul 15 16:11 .config
drwxr-xr-x 4 root root 4096 Jul 22 07:49 kgan
drwxr-xr-x 1 root root 4096 Jul 10 16:29 sample_data


# Set the root directory.

In [3]:
import os

root_dir = '/content/kgan'
os.chdir(root_dir)

!ls -al

total 36
drwxr-xr-x 4 root root  4096 Jul 22 07:49 .
drwxr-xr-x 1 root root  4096 Jul 22 07:49 ..
drwxr-xr-x 8 root root  4096 Jul 22 07:49 .git
-rw-r--r-- 1 root root  1799 Jul 22 07:49 .gitignore
drwxr-xr-x 5 root root  4096 Jul 22 07:49 kgan
-rw-r--r-- 1 root root 11357 Jul 22 07:49 LICENSE
-rw-r--r-- 1 root root    58 Jul 22 07:49 README.md


In [4]:
!git pull origin

Already up to date.


# Import TensorFlow 2.x.

In [5]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.random.set_seed(7)

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

2.2.0


# Import TensorFlow datasets.

In [11]:
import tensorflow_datasets as tfds

# Load MNIST dataset.

### Load MNIST dataset.

In [12]:
train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


### Normalize dataset images.

In [13]:
def _normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.
    return (image, label)

### Create dataset batches.

In [14]:
buffer_size = 1024
batch_size = 32

In [15]:
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
train_dataset = train_dataset.map(_normalize_image)

test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(_normalize_image)

# Create optimizer.

In [16]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Import model factory.

In [6]:
from kgan.models.ModelFactory import ModelFactory

# Create discriminator model.

In [7]:
image_shape = (28, 28, 1)

In [8]:
discriminator = ModelFactory.create_discriminator('gan', image_shape)

In [17]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Create generator model.

In [9]:
latent_dimension = 100
noise_shape = (latent_dimension,)

In [10]:
generator = ModelFactory.create_generator('gan', noise_shape, image_shape)

In [18]:
generator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Create GAN model.

In [19]:
input_images = layers.Input(shape=noise_shape)
generated_images = generator(input_images)

In [20]:
discriminator.trainable = False
predictions = discriminator(generated_images)

In [21]:
gan = models.Model(input_images, predictions, name='gan')
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])